In [38]:
"""--------------------------------------------------------------------------------
                        MODULES
----------------------------------------------------------------------------------"""
# ---- MongoDb
import pymongo
from bson.objectid import ObjectId
# ---- DataManipulation
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
# ---- ConnectSQL 
import pyodbc
# ---- Plot 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ---- DateManipulation
import datetime as dt
import time
# ---- System
import sys
# ---- JSON
import json
# ---- Translate
import googletrans
from googletrans import Translator

In [39]:
"""--------------------------------------------------------------------------------
                        LOAD FILES
----------------------------------------------------------------------------------"""
# ---- LoadFrame
french_frame = pd.read_csv("fr.csv")
dutch_frame = pd.read_csv("nl.csv")
english_frame = pd.read_csv("en.csv")

# ---- CountValues
french_counts = french_frame.intent.value_counts().rename_axis('unique_values').to_frame('counts').sort_values(by='unique_values').reset_index()
dutch_counts = dutch_frame.intent.value_counts().rename_axis('unique_values').to_frame('counts').sort_values(by='unique_values').reset_index()
english_counts = english_frame.intent.value_counts().rename_axis('unique_values').to_frame('counts').sort_values(by='unique_values').reset_index()

# ---- Total frame
total_figures = pd.merge(french_counts,dutch_counts,how = "left",left_on = 'unique_values',right_on='unique_values',)  #drop('user_id',axis=1) .rename(columns={"LoginID": "created_week"})
total_figures = pd.merge(total_figures,english_counts,how = "left",left_on = 'unique_values',right_on='unique_values',).rename(columns={"counts": "english","counts_x": "french","counts_y": "dutch"}).fillna(0)

# 1. No expressions

In [40]:
missing_frame = total_figures[ ((total_figures['french'] == 0) |(total_figures['english'] == 0) | (total_figures['dutch']== 0) )]

# 2. Too few expressions

In [41]:
total_figures[ ((total_figures['french'] < 20) |(total_figures['english'] < 20) | (total_figures['dutch']< 20) )]

,unique_values,french,dutch,english
1,Adm.A&S.SetupYourDomainNameIdentify,34,19.0,31.0
14,Adm.ContSourc.CreateContactSource,26,18.0,26.0
27,Adm.Mail.EditTemplate,26,10.0,25.0
30,Adm.ProspStat.CreateProspectionStatus,26,18.0,26.0
38,Adm.lab.CreateLabel,29,1.0,29.0
39,Cal.App.AddHistoryItem,129,0.0,129.0
53,Company.General.Create,51,12.0,48.0
67,Cont.Gen.AddProptertyTo,112,11.0,113.0
86,Help.CallWhise,32,1.0,29.0
87,Help.CopyPasteText,23,2.0,23.0


In [42]:
translator = Translator()
result = translator.translate('Mikä on nimesi', src='fi', dest='fr')

print(result.src)
print(result.dest)
print(result.text)

fi
fr
Quel est votre nom


# Translate Missing

In [43]:
LANG = 'dutch'
# ---- Get The expressions to translate
translate_frame = french_frame[french_frame.intent.isin(list(total_figures[ total_figures[LANG] == 0].unique_values))]
# ---- Translate them in one go
if LANG == 'dutch':
    result = translator.translate(list(translate_frame.expression), src='fr', dest='nl')
elif LANG == 'english':
    result = translator.translate(list(translate_frame.expression), src='fr', dest='en')
# ---- Add to the frame
result = [k.text for k in result]
translate_frame[LANG] = result
# ---- Save
translate_frame.to_excel("translation_missing"+LANG+'.xlsx')  
translate_frame

,intent,expression,dutch
2560,Umail.Mail.AddMailToHistory,Comment ajouter un email à un historique,Hoe u een e-mail aan een geschiedenis toevoegt
2561,Umail.Mail.AddMailToHistory,Comment attacher un email à un historique,Hoe u een e-mail bij een geschiedenis kunt voegen
2562,Umail.Mail.AddMailToHistory,Comment créer un historique à partir d'un email,Een geschiedenis maken vanuit een e-mail
2563,Umail.Mail.AddMailToHistory,Comment garder un trace des mails envoyés à un...,Hoe u e-mails kunt bijhouden die naar een kand...
2564,Umail.Mail.AddMailToHistory,Comment garder un trace des mails envoyés à un...,Hoe u e-mails kunt bijhouden die naar een klan...
2565,Umail.Mail.AddMailToHistory,Comment garder un trace des mails envoyés à un...,Hoe u e-mails kunt bijhouden die naar een cont...
2566,Umail.Mail.AddMailToHistory,Comment garder un trace des mails envoyés à un...,Hoe u e-mails kunt bijhouden die naar een eige...
2567,Umail.Mail.AddMailToHistory,Comment intégrer un email à un historique,Hoe een e-mail in een geschiedenis te integreren
2568,Umail.Mail.AddMailToHistory,Comment joindre un email à un historique,Hoe u een e-mail bij een geschiedenis kunt voegen
2569,Umail.Mail.AddMailToHistory,Comment lier un email à un historique,Hoe u een e-mail aan een geschiedenis koppelt
